In [3]:
import sys

sys.path.insert(0, './ModelCompression')
dataset = 'mnist'
current_state = 'layer_input'
next_state = 'layer_output'

In [4]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import logging
import pandas as pd
from IPython.display import clear_output
from tqdm import tqdm
from CompressionLibrary.utils import *
from CompressionLibrary.CompressionTechniques import *
from CompressionLibrary.replay_buffer import ReplayBuffer
from CompressionLibrary.environments import *
from CompressionLibrary.custom_layers import ROIEmbedding, ROIEmbedding1D
from CompressionLibrary.reinforcement_models import DQNAgent
import sys
import gc

logging.basicConfig(level=logging.INFO, handlers=[
        logging.FileHandler('./data/ModelCompression.log')],
        # logging.StreamHandler()], 
        format='%(asctime)s -%(levelname)s - %(funcName)s -  %(message)s')

# Evaluate agent

In [5]:
def evaluate_adadeep(env, conv_agent, fc_agent, n_games=1, greedy=True, t_max=10000):
    """ Plays n_games full games. If greedy, picks actions as argmax(qvalues). Returns mean reward. """
    rewards = []
    acc = []
    weights = []
    infos = []
    for _ in range(n_games):
        s = env.reset()
        reward = 0
        df = pd.DataFrame()
        for k in range(len(env.layer_name_list)):
            next_layer_name = env.layer_name_list[env._layer_counter]
            layer = env.model.get_layer(next_layer_name)
            was_conv = False
            random_images = s[np.random.choice(s.shape[0], size=128)]
            if isinstance(layer, tf.keras.layers.Conv2D):
                # qvalues = conv_agent.get_qvalues(random_image).numpy()
                # action = conv_agent.sample_actions(qvalues)[0]
                qvalues = conv_agent.get_qvalues(random_images).numpy()
                action = conv_agent.sample_actions_using_mode(qvalues)[0]
                was_conv = True
            if isinstance(layer, tf.keras.layers.Dense):
                qvalues = fc_agent.get_qvalues(random_images).numpy()
                action = fc_agent.sample_actions_using_mode(qvalues)[0]

            new_s, r, done, info = env.step(action)
            s = env.get_state('current_state')
            if done:
                s = env.reset()
                break

            row = {'state': s, 'action': action, 'reward': r,
                   'next_state': new_s, 'done': done, 'info': info}
            df = df.append(row, ignore_index=True)

        # Calculate reward using stats before and after compression
        before_stats = df.iloc[0]['info']
        after_stats = df.iloc[-1]['info']
        reward = 1 - after_stats['weights_after'] / \
            before_stats['weights_before'] + after_stats['acc_after']

        rewards.append(reward)
        acc.append(info['acc_after'])
        weights.append(info['weights_after'])
    return np.mean(rewards), np.mean(acc), np.mean(weights)

# Create environment

In [6]:
def make_env(dataset, current_state_source='layer_input', next_state_source='layer_output'):
    train_ds, valid_ds, test_ds, input_shape, num_classes = load_dataset(dataset)

    model_path = './data/full_model/test_'+dataset
    try:
      model = tf.keras.models.load_model(model_path, compile=True)
    except OSError:
      optimizer = tf.keras.optimizers.Adam()
      loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
      train_metric = tf.keras.metrics.SparseCategoricalAccuracy()
      model = tf.keras.Sequential([tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_0',
                                                          input_shape=input_shape),
                                    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='conv2d_1'),
                                    tf.keras.layers.MaxPool2D((2, 2), 2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation='relu', name='dense_0'),
                                    tf.keras.layers.Dense(128, activation='relu', name='dense_1'),
                                    tf.keras.layers.Dense(num_classes, activation='softmax', name='dense_softmax')
                                    ])
      model.compile(optimizer=optimizer, loss=loss_object, metrics=train_metric)
      model.fit(train_ds, epochs=5, validation_data=valid_ds)
      model.save(model_path)


    w_comprs = ['InsertDenseSVD', 'InsertDenseSparse', 'DeepCompression'] # 'InsertDenseSVDCustom'
    c_comprs = ['InsertSVDConv', 'SparseConvolutionCompression','DepthwiseSeparableConvolution', 'SparseConnectionsCompression'] 
    l_comprs = ['FireLayerCompression', 'MLPCompression','ReplaceDenseWithGlobalAvgPool']
    compressors_list = w_comprs + c_comprs + l_comprs

    parameters = {}
    parameters['DeepCompression'] = {'layer_name': 'dense_0', 'threshold': 0.001}
    parameters['ReplaceDenseWithGlobalAvgPool'] = {'layer_name': 'dense_1'}
    parameters['InsertDenseSVD'] = {'layer_name': 'dense_0', 'units': 16}
    parameters['InsertDenseSVDCustom'] = {'layer_name': 'dense_0', 'units': 16}
    parameters['InsertDenseSparse'] = {'layer_name': 'dense_0', 'verbose': True, 'units': 16}
    parameters['InsertSVDConv'] = {'layer_name': 'conv2d_1', 'units': 8}
    parameters['DepthwiseSeparableConvolution'] = {'layer_name': 'conv2d_1'}
    parameters['FireLayerCompression'] = {'layer_name': 'conv2d_1'}
    parameters['MLPCompression'] = {'layer_name': 'conv2d_1'}
    parameters['SparseConvolutionCompression'] = {'layer_name': 'conv2d_1', 'bases':4}
    parameters['SparseConnectionsCompression'] = {'layer_name': 'conv2d_1', 'epochs': 20,
                                                  'target_perc': 0.75, 'conn_perc_per_epoch': 0.1}

    layer_name_list = ['conv2d_1','dense_0', 'dense_1']

    env = ModelCompressionEnv(compressors_list, model_path, parameters,
                 train_ds, valid_ds, test_ds,
                 layer_name_list, input_shape, current_state_source=current_state_source, next_state_source=next_state_source, verbose=False)
    
    return env

env = make_env(dataset, current_state, next_state)
env.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_0 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense_0 (Dense)              (None, 128)               589952    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_softmax (Dense)        (None, 10)                1

# Create DQN for model compression


In [7]:
fc_state_dim = (1,)
fc_n_actions = len(env.dense_compressors)
conv_state_dim = list(env.get_state('current_state').shape)[1:]
print(conv_state_dim)
conv_n_actions = len(env.conv_compressors)

fc_agent = DQNAgent("dqn_agent_fc", fc_state_dim, fc_n_actions, epsilon=0.9, layer_type='fc')
fc_target_network = DQNAgent("target_network_fc", fc_state_dim, fc_n_actions, layer_type='fc')


conv_agent = DQNAgent("dqn_agent_conv", conv_state_dim, conv_n_actions, epsilon=0.9, layer_type='cnn')
conv_target_network = DQNAgent("target_network_conv", conv_state_dim, conv_n_actions, layer_type='cnn')

try:
    conv_target_network.model.load_weights(
        './data/checkpoints/{}_my_checkpoint_conv'.format(dataset))
    fc_target_network.model.load_weights(
        './data/checkpoints/{}_my_checkpoint_fc'.format(dataset))
except:
    raise

[26, 26, 32]
Model: "dqn_agent_fc"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None, 1)]         0         
_________________________________________________________________
roi_embedding1d (ROIEmbeddin (None, 63)                0         
_________________________________________________________________
dense (Dense)                (None, 512)               32768     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 2052      
Total params: 34,820
Trainable params: 34,820
Non-trainable params: 0
_________________________________________________________________
Model: "target_network_fc"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, None, 1)]         0         
__________

# Training func


In [8]:



evaluate_adadeep(make_env(dataset), conv_agent, fc_agent, n_games=10)

KeyboardInterrupt: 